"""
Odyssey Protocol: Xylos-7 Expedition

Story Overview:
The year is 2342. Humanity's ambitious deep-space colonization effort, the 'Odyssey Prime' colony ship, has
vanished without a trace after entering the Kepler-186f equivalent, now designated 'Xylos-7'. Weeks of
silence were broken by a faint, heavily encrypted beacon signal emanating from the planet's surface.

You are Commander [Your Name], assigned to the lead exploratory drone unit 'Pathfinder-1'. Your mission:
to navigate Xylos-7's notoriously treacherous and dynamic terrain, reach the anomalous Beacon Source,
and retrieve its encrypted data before the planet's hostile environment claims the only remaining clue
to the Odyssey Prime's fate.

Xylos-7 is a world of extremes: vast, impassable ocean abysses, volatile volcanic craters, treacherous
rocky formations, and atmospheric anomalies like dense fogs and disruptive magnetic fields. Even seemingly
solid ground can be unstable or infused with corrosive elements. Your drone must meticulously analyze
each grid sector, identifying traversal costs and impassable hazards, to forge a path through the
uncharted dangers.

The future of the Odyssey Prime, and potentially humanity's expansion, rests on your ability to master
this alien landscape.
"""

In [8]:
import numpy as np
import random
import time
import math

# --- Global Constants for Traversal Cost Analysis ---
MIN_STABILITY_FOR_TRAVERSAL = 0.5              # Min stability for passage (below this, cost is infinity)
MAX_WATER_COMPOSITION_FOR_TRAVERSAL = 0.3      # Max water for passage (above this, cost is infinity)
MAX_SOLID_COMPOSITION_FOR_TRAVERSAL = 0.4      # Max (rock + silicates) for passage (above this, cost is infinity)
                                               # (Note: Renamed from MIN_SOLID_COMPOSITION_FOR_TRAVERSAL for clarity)

DEFAULT_TRAVERSAL_COST = 1.0                   # Base cost for easily traversable terrain
DIFFICULT_TERRAIN_COST = 5.0                   # Example higher cost for difficult but traversable terrain
IMPASSABLE_COST = math.inf                     # Represents infinite cost for impassable terrain
# --- End Global Constants ---

In [9]:
class Block:
    """Base class for all terrain features, calculating a numerical traversal cost."""
    def __init__(self, name, description,
                 geological_properties=None, stability=0.0, is_obstacle=False):
        self.name = name
        self.description = description
        self.geological_properties = geological_properties if geological_properties is not None else {
            'rock': 0.0, 'silicates': 0.0, 'water': 1.0, 'carbon': 0.0
        }
        self.stability = stability
        self.is_obstacle = is_obstacle
        self.details = f"Details: {description}"

    def get_traversal_cost(self):
        """
        Calculates the numerical cost to traverse this terrain.
        Returns float('inf') if impassable.
        """
        # Condition 1: Direct obstacle check - always impassable
        if self.is_obstacle:
            return IMPASSABLE_COST

        # Condition 2: Stability check - too low stability makes it impassable
        if self.stability < MIN_STABILITY_FOR_TRAVERSAL:
            return IMPASSABLE_COST

        # Condition 3: Water composition check - too much water makes it impassable
        if self.geological_properties.get('water', 0.0) > MAX_WATER_COMPOSITION_FOR_TRAVERSAL:
            return IMPASSABLE_COST

        # Condition 4: Solid composition (rock + silicates) check - too much solid makes it impassable
        # The logic here makes 'too much solid' (> MAX_SOLID_COMPOSITION_FOR_TRAVERSAL) impassable.
        solid_comp = self.geological_properties.get('rock', 0.0) + self.geological_properties.get('silicates', 0.0)
        if solid_comp > MAX_SOLID_COMPOSITION_FOR_TRAVERSAL:
            return IMPASSABLE_COST
        
        # If all impassability checks pass, return a default traversable cost
        return DEFAULT_TRAVERSAL_COST

    def __repr__(self):
        cost = self.get_traversal_cost()
        if cost == IMPASSABLE_COST:
            status = "IMPASSABLE"
        else:
            status = f"Cost:{cost:.1f}"
        return f"{self.name}({status}, Stab:{self.stability:.1f})"

# --- Specific Terrain Types (Adjusted for Traversal Costs) ---

class FlatGround(Block):
    """Basic, easily traversable ground."""
    def __init__(self):
        super().__init__("Flat Ground", "Solid, even, and clear ground.",
                         geological_properties={
                             'rock': random.uniform(0.10, 0.20),
                             'silicates': random.uniform(0.05, 0.10), # Total solid: 0.15 - 0.30 (<= 0.4 OK)
                             'water': random.uniform(0.0, 0.10),      # Water: 0.0 - 0.10 (<= 0.3 OK)
                             'carbon': random.uniform(0.0, 0.05)
                         },
                         stability=random.uniform(0.8, 1.0),         # Stability: 0.8 - 1.0 (>= 0.5 OK)
                         is_obstacle=False)                           # Not a direct obstacle
    # Intended Cost: DEFAULT_TRAVERSAL_COST (1.0)

class CrumbledPath(Block):
    """Ground that is traversable but difficult due to unevenness."""
    def __init__(self):
        super().__init__("Crumbled Path", "Uneven and difficult ground.",
                         geological_properties={
                             'rock': random.uniform(0.20, 0.30),
                             'silicates': random.uniform(0.10, 0.15), # Total solid: 0.30 - 0.45 (can be > 0.4 making it impassable *or* higher cost if logic changes)
                             'water': random.uniform(0.10, 0.20),      # Water: 0.10 - 0.20 (<= 0.3 OK)
                             'carbon': random.uniform(0.05, 0.10)
                         },
                         stability=random.uniform(0.6, 0.7),         # Stability: 0.6 - 0.7 (>= 0.5 OK)
                         is_obstacle=False)
    # With current get_traversal_cost: This could be IMPASSABLE if solid_comp > 0.4, or DEFAULT_TRAVERSAL_COST if <= 0.4.
    # To make it DIFFICULT_TERRAIN_COST, the get_traversal_cost logic would need to be expanded.
    # For now, it's either 1.0 or inf.

class BeaconSource(Block):
    """The mysterious beacon emitting the signal."""
    def __init__(self, encoded_data, name="Beacon Source"):
        super().__init__(name, "The origin of the signal.",
                         geological_properties={
                             'rock': random.uniform(0.10, 0.20),
                             'silicates': random.uniform(0.05, 0.10), # Total solid: 0.15 - 0.30 (<= 0.4 OK)
                             'water': random.uniform(0.0, 0.05),      # Water: 0.0 - 0.05 (<= 0.3 OK)
                             'carbon': random.uniform(0.0, 0.05)
                         },
                         stability=random.uniform(0.9, 1.0),         # Stability: 0.9 - 1.0 (>= 0.5 OK)
                         is_obstacle=False)
        self.encoded_discovery_data = encoded_data
    # Intended Cost: DEFAULT_TRAVERSAL_COST (1.0)


# --- Non-Traversable / Obstacle Terrain Types ---
class EmptySpace(Block):
    """Vast, deep space / unchartable abyss."""
    def __init__(self):
        super().__init__("Ocean Abyss", "A void where nothing can exist.",
                         geological_properties={'rock': 0, 'silicates': 0, 'water': 1.0, 'carbon': 0}, # Water 1.0 (> 0.3 FAIL)
                         stability=0.0,                                                             # Stability 0.0 (< 0.5 FAIL)
                         is_obstacle=False)
    # Intended Cost: IMPASSABLE_COST

class RockyTerrain(Block):
    """Solid, high-density rock formations, too dense to traverse."""
    def __init__(self, name="Rocky Terrain"):
        super().__init__(name, "Dense, ancient rock formations.",
                         geological_properties={
                             'rock': random.uniform(0.6, 0.9),
                             'silicates': random.uniform(0.05, 0.2), # Total solid: 0.65 - 1.10 (> 0.4 FAIL)
                             'water': random.uniform(0, 0.1),
                             'carbon': random.uniform(0, 0.05)
                         },
                         stability=random.uniform(0.7, 1.0),
                         is_obstacle=False)
    # Intended Cost: IMPASSABLE_COST

class VolcanicCrater(Block):
    """Highly unstable, molten areas."""
    def __init__(self):
        super().__init__("Volcanic Crater", "A bubbling, molten pit.",
                         geological_properties={'rock': 0.1, 'silicates': 0.1, 'water': 0.4, 'carbon': 0.0},
                         stability=random.uniform(0.0, 0.4), # Stability: 0.0 - 0.4 (< 0.5 FAIL)
                         is_obstacle=False)
    # Intended Cost: IMPASSABLE_COST

class HydrothermalVent(Block):
    """Underwater/subsurface hot vents, too watery/unstable."""
    def __init__(self):
        super().__init__("Hydrothermal Vent", "Geothermal activity creating hot, unstable water flows.",
                         geological_properties={'rock': 0.05, 'silicates': 0.05, 'water': random.uniform(0.7, 0.9), 'carbon': 0.1}, # Water: 0.7 - 0.9 (> 0.3 FAIL)
                         stability=random.uniform(0.1, 0.3),                                                                       # Stability: 0.1 - 0.3 (< 0.5 FAIL)
                         is_obstacle=False)
    # Intended Cost: IMPASSABLE_COST

# --- Explicit Obstacle Types (Always Impassable by definition) ---
class DenseFog(Block):
    """Thick atmospheric conditions, blocking all sensors and traversal."""
    def __init__(self):
        super().__init__("Dense Fog", "An atmospheric anomaly, impenetrable.",
                         geological_properties={'rock': 0, 'silicates': 0, 'water': 1, 'carbon': 0},
                         stability=0.0,
                         is_obstacle=True) # Direct obstacle (FAIL)
    # Intended Cost: IMPASSABLE_COST

class MagneticAnomaly(Block):
    """Intense magnetic fields disrupting all systems."""
    def __init__(self):
        super().__init__("Magnetic Anomaly", "A localized magnetic field, disrupts all electronics.",
                         geological_properties={}, stability=0.0, is_obstacle=True) # Direct obstacle (FAIL)
    # Intended Cost: IMPASSABLE_COST

class EnergyBarrier(Block):
    """An unknown, static energy field."""
    def __init__(self):
        super().__init__("Energy Barrier", "A shimmering, impassable energy wall.",
                         geological_properties={}, stability=0.0, is_obstacle=True) # Direct obstacle (FAIL)
    # Intended Cost: IMPASSABLE_COST

In [10]:
#Scanning Landing zone to prepare base for makeshift HQ. To analyze envirnoment and make a plan for reaching the beacon
# Define ANSI escape codes for formatting
BOLD = "\033[1m"
RESET = "\033[0m"
GREEN = "\033[92m" # For success messages
YELLOW = "\033[93m" # For warnings/status updates
BLUE = "\033[94m" # For general info
MAGENTA = "\033[95m" # For analytical data
RED = "\033[91m"

print(f"{BLUE}--- ODYSSEY PROTOCOL: PATHFINDER-1 DEPLOYMENT ---{RESET}\n")
time.sleep(1)

print(f"{YELLOW}[07:00:00 ZULU] {RESET}Undocking sequence initiated. Pathfinder-1 is separating from the primary 'Odyssey Prime' carrier vessel. All clamps disengaged.")
time.sleep(2)
print(f"{YELLOW}[07:00:15 ZULU] {RESET}Umbilical severance confirmed. Pathfinder-1 is now independent. Thrusters engaging for orbital insertion.")
time.sleep(2)

print(f"\n{BLUE}--- ATMOSPHERIC ENTRY PROTOCOL ---{RESET}")
print(f"{YELLOW}[07:15:00 ZULU] {RESET}Approach vector Xylos-7 established. Initial atmospheric friction heating detected. {BOLD}External temperatures rising.{RESET}")
time.sleep(2)
print(f"{YELLOW}[07:16:00 ZULU] {RESET}Entering Upper Exosphere (Layer 5). {MAGENTA}Atmospheric Analysis (3D Array Challenge Hint):{RESET}")
print(f"{MAGENTA}  - Altitude: ~100 km. Pressure: 0.001 Pa. Temperature: -70°C. Primary Composition: Hydrogen (90%), Helium (9%).{RESET}")
time.sleep(2)
print(f"{YELLOW}[07:17:30 ZULU] {RESET}Crossing Mesospheric boundary (Layer 4). {BOLD}Gravitational forces immense.{RESET} Structural integrity holding. Compensators at 85%.")
print(f"{MAGENTA}  - Altitude: ~50 km. Pressure: 100 Pa. Temperature: -20°C. Primary Composition: Nitrogen (70%), Methane (25%), Trace Argon.{RESET}")
time.sleep(2)
print(f"{YELLOW}[07:18:45 ZULU] {RESET}Lower Stratosphere entered (Layer 3). Activating aerofoil stabilizers. {BOLD}Minor system fluctuations detected from localized magnetic distortions.{RESET}")
print(f"{MAGENTA}  - Altitude: ~20 km. Pressure: 5000 Pa. Temperature: 5°C. Primary Composition: Nitrogen (65%), Methane (20%), Oxygen (10%), Water Vapor (5%).{RESET}")
time.sleep(2)
print(f"{YELLOW}[07:19:10 ZULU] {RESET}Penetrating Troposphere (Layer 2). Atmospheric density increasing. Visual spectrum obscured by dense cloud cover. Relying on sub-surface radar and active sonar for guidance.")
print(f"{MAGENTA}  - Altitude: ~5 km. Pressure: 50,000 Pa. Temperature: 15°C. Primary Composition: Nitrogen (60%), Oxygen (20%), Water Vapor (15%), Trace Carbon Compounds.{RESET}")
time.sleep(2)

print(f"\n{BLUE}--- LANDING ZONE RECONNAISSANCE ---{RESET}")
print(f"{YELLOW}[07:25:00 ZULU] {RESET}Hovering at 500 meters above designated landing coordinates ({start_coords[0]}, {start_coords[1]}). Initiating comprehensive land scan analysis.")
time.sleep(3)

# Get the actual object at start_coords from your live grid
# ENSURE gridSector_Delta AND start_coords ARE POPULATED IN YOUR MAIN SCRIPT
landing_site_block = gridSector_Delta[start_coords[0]][start_coords[1]]
if landing_site_block:
    print(f"{MAGENTA}  - Landing Site Scan Details: {RESET}")
    print(f"{MAGENTA}    Object Type: {landing_site_block.name}{RESET}")
    print(f"{MAGENTA}    Description: {landing_site_block.description}{RESET}")
    print(f"{MAGENTA}    Calculated Traversal Cost: {landing_site_block.get_traversal_cost():.1f}{RESET}") # Display cost
    if landing_site_block.get_traversal_cost() == IMPASSABLE_COST:
        print(f"{RED}{BOLD}    WARNING: Landing site is IMPASSABLE! Recommending immediate abort or rerouting.{RESET}")
    else:
        print(f"{GREEN}    STATUS: Landing site traversable. Proceeding with final descent.{RESET}")
else:
    print(f"{YELLOW}  - Landing Site Scan Details: UNKNOWN (Grid data not fully populated at {start_coords}). Proceeding with caution.{RESET}")
time.sleep(2)


print(f"{YELLOW}[07:27:30 ZULU] {RESET}Landing thrusters modulating. Altitude: 100 meters. Dust storm generating from local seismic activity. Visibility: Poor.")
time.sleep(2)
print(f"{GREEN}[07:28:00 ZULU] {RESET}{BOLD}Touchdown confirmed! Pathfinder-1 is safely on Xylos-7's surface.{RESET}")
time.sleep(1)

print(f"\n{BLUE}--- MAKESHIFT HQ DEPLOYMENT ---{RESET}")
print(f"{YELLOW}[07:30:00 ZULU] {RESET}Cargo bay doors opening. Automated systems deploying prefabricated modules and resource extractors.")
time.sleep(3)
print(f"{YELLOW}[07:32:15 ZULU] {RESET}Initial power conduits established. Commencing ground stabilization for main habitat unit.")
time.sleep(2)
print(f"{GREEN}[07:35:00 ZULU] {RESET}{BOLD}Makeshift HQ operational.{RESET} Local atmospheric readings stable. Basic life support initiated.")
time.sleep(1)

print(f"\n{BLUE}--- BEACON RETRIEVAL PLANNING ---{RESET}")
print(f"{YELLOW}[07:45:00 ZULU] {RESET}Primary objective re-affirmed: Locate and access Beacon Source. Analyzing optimal traversal routes from HQ.")
time.sleep(2)
print(f"{YELLOW}[07:46:30 ZULU] {RESET}Initial pathfinding models suggest challenging terrain. Deploying specialized vehicles: 'Harvester' (rugged terrain), 'Aqua-Rover' (waterways), and 'Sky-Scout' (aerial survey).")
time.sleep(2)
print(f"{YELLOW}[07:48:00 ZULU] {RESET}Command, Pathfinder-1 standing by for orders. Preparing for next phase of beacon retrieval operation.")
print(f"{BLUE}--- END LOG ---{RESET}")

--- ODYSSEY PROTOCOL: PATHFINDER-1 DEPLOYMENT ---

[07:00:00 ZULU] Undocking sequence initiated. Pathfinder-1 is separating from the primary 'Odyssey Prime' carrier vessel. All clamps disengaged.
[07:00:15 ZULU] Umbilical severance confirmed. Pathfinder-1 is now independent. Thrusters engaging for orbital insertion.

--- ATMOSPHERIC ENTRY PROTOCOL ---
[07:15:00 ZULU] Approach vector Xylos-7 established. Initial atmospheric friction heating detected. External temperatures rising.
[07:16:00 ZULU] Entering Upper Exosphere (Layer 5). Atmospheric Analysis (3D Array Challenge Hint):
  - Altitude: ~100 km. Pressure: 0.001 Pa. Temperature: -70°C. Primary Composition: Hydrogen (90%), Helium (9%).
[07:17:30 ZULU] Crossing Mesospheric boundary (Layer 4). Gravitational forces immense. Structural integrity holding. Compensators at 85%.
  - Altitude: ~50 km. Pressure: 100 Pa. Temperature: -20°C. Primary Composition: Nitrogen (70%), Methane (25%), Trace Argon.
[07:18:45 ZULU] Lower Stratosphere enter

In [11]:
PLANET_GRID_SIZE = 10 # Increased size for a better hazardous feel
gridSector_Delta = np.empty((PLANET_GRID_SIZE, PLANET_GRID_SIZE), dtype=object)

beacon_coords = (random.randint(0, PLANET_GRID_SIZE - 1), random.randint(0, PLANET_GRID_SIZE - 1))
start_coords = (random.randint(0, PLANET_GRID_SIZE - 1), random.randint(0, PLANET_GRID_SIZE - 1))

# Ensure start and beacon are not the same (optional but good practice)
while start_coords == beacon_coords:
    start_coords = (random.randint(0, PLANET_GRID_SIZE - 1), random.randint(0, PLANET_GRID_SIZE - 1))

for r in range(PLANET_GRID_SIZE):
    for c in range(PLANET_GRID_SIZE):
        # Prioritize placing the specific beacon and start points
        if (r, c) == beacon_coords:
            gridSector_Delta[r, c] = BeaconSource("alpha_signal_data")
        elif (r, c) == start_coords:
            gridSector_Delta[r, c] = FlatGround() # Ensure start is traversable
        else:
            # Roll the dice for various hazard types, making FlatGround scarce
            hazard_roll = random.random()

            if hazard_roll < 0.20: # 20% chance for Rocky Terrain
                gridSector_Delta[r, c] = RockyTerrain()
            elif hazard_roll < 0.35: # Next 15% (total 35%) chance for Ocean Abyss (EmptySpace)
                gridSector_Delta[r, c] = EmptySpace()
            elif hazard_roll < 0.45: # Next 10% (total 45%) chance for Volcanic Crater
                gridSector_Delta[r, c] = VolcanicCrater()
            elif hazard_roll < 0.55: # Next 10% (total 55%) chance for Hydrothermal Vent
                gridSector_Delta[r, c] = HydrothermalVent()
            elif hazard_roll < 0.65: # Next 10% (total 65%) chance for Dense Fog
                gridSector_Delta[r, c] = DenseFog()
            elif hazard_roll < 0.70: # Next 5% (total 70%) chance for Magnetic Anomaly
                gridSector_Delta[r, c] = MagneticAnomaly()
            elif hazard_roll < 0.75: # Next 5% (total 75%) chance for Energy Barrier
                gridSector_Delta[r, c] = EnergyBarrier()
            else: # Remaining 25% chance for FlatGround (your scarce islands)
                gridSector_Delta[r, c] = FlatGround()

print(f"Beacon located at: {beacon_coords}")
print(f"Drone starting at: {start_coords}")
print("\nGrid generation complete. Your mission parameters are set for a challenging exploration.")

Beacon located at: (3, 4)
Drone starting at: (6, 6)

Grid generation complete. Your mission parameters are set for a challenging exploration.


In [12]:
import sys
def typeText (string, delay=0.05):
    for char in string:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)

print('-' *40)
print(f"{BOLD}{GREEN}--- Sector {(50, 50)} ---{RESET}")
print(f"| {MAGENTA} Type:{RESET}: Flat Ground")
print(f"| {MAGENTA}Desc:{RESET}: Solid, even, and clear ground.")
print(f"| {MAGENTA}Status:{RESET}: {GREEN}Cost: 1.0{RESET}")
print(f"| {MAGENTA}Statibility:{RESET}: 0.89 (Required > 0.50)")
print(f"| {MAGENTA}Obstacle:{RESET}: False")
print(f"| {MAGENTA}Geology:{RESET}:")
print(f"|     - {MAGENTA}Rock:{RESET}: 1")
print(f"|     - {MAGENTA}Silicates:{RESET}: 0.09")
print(f"|     - {MAGENTA}Rock:{RESET}: 0.12")
print(f"|     - {MAGENTA}Rock:{RESET}: 1")
print('-' *40)


----------------------------------------
--- Sector (50, 50) ---
|  Type:: Flat Ground
| Desc:: Solid, even, and clear ground.
| Status:: Cost: 1.0
| Statibility:: 0.89 (Required > 0.50)
| Obstacle:: False
| Geology::
|     - Rock:: 1
|     - Silicates:: 0.09
|     - Rock:: 0.12
|     - Rock:: 1
----------------------------------------


In [14]:
BOLD = "\033[1m"
RESET = "\033[0m"
GREEN = "\033[92m"   # For success messages
YELLOW = "\033[93m"  # For warnings/status updates
BLUE = "\033[94m"   # For general info
MAGENTA = "\033[95m" # For analytical data
CYAN = "\033[96m"    # For vehicle status
RED = "\033[91m"    # For critical warnings/impassable
GRAY = "\033[90m"   # For subtle background info

def type_text(text, delay=0):
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    sys.stdout.write('\n')
type_text(f"{BLUE}--- POST-LAND OPERATIONS: GRID MAPPING INITIATED ---{RESET}\n")
time.sleep(1.0)
type_text(f"{YELLOW}[07:50:00 ZULU] {RESET}Makeshift HQ fully established. Commencing high-resolution grid mapping.")
time.sleep(1.0)
type_text(f"{CYAN}[07:51:00 ZULU] {RESET}Deploying 'Harvester' ground drone for Sector A. 'Sky-Scout' aerial unit deploying for Sector B overview. 'Aqua-Rover' standing by for hydrological mapping.")
time.sleep(1.5)
type_text(f"{YELLOW}[07:55:00 ZULU] {RESET}Establishing high-bandwidth uplink to 'Odyssey Prime' Mother Vessel for real-time data synchronization. {BOLD}Mother Vessel acknowledging data stream.{RESET}")
time.sleep(1.0)

type_text(f"\n{BLUE}--- DETAILED GRID ACQUISITION REPORT ({PLANET_GRID_SIZE}x{PLANET_GRID_SIZE}) ---{RESET}")
type_text(f"{MAGENTA}Scanning and populating grid with detailed environmental data. Stand by for extensive data log...{RESET}\n")
time.sleep(2.0)

total_cells = PLANET_GRID_SIZE * PLANET_GRID_SIZE
cells_scanned = 0
impassable_count = 0
traversable_count = 0
terrain_type_counts = {}

# Use a slightly faster delay for individual property lines to avoid excessive wait times
detail_char_delay = 0.005

for r in range(PLANET_GRID_SIZE):
    for c in range(PLANET_GRID_SIZE):
        current_block = gridSector_Delta[r, c]
        cells_scanned += 1

        # Update progress bar
        progress_percent = (cells_scanned / total_cells) * 100
        bar = '#' * int(progress_percent / 5)
        sys.stdout.write(f"\r{GRAY}Scan Progress: [{bar:20}] {progress_percent:.1f}% Complete{RESET}")
        sys.stdout.flush()

        # Ensure we have a TerrainObject instance
        if not isinstance(current_block, RockyTerrain):
            type_text(f"\n({r:02}, {c:02}): {RED}ERROR - Unknown Object Type! Skipping detailed scan.{RESET}", detail_char_delay)
            continue

        cost = current_block.get_traversal_cost()
        if cost == IMPASSABLE_COST:
            impassable_count += 1
        else:
            traversable_count += 1

        terrain_type_counts[current_block.name] = terrain_type_counts.get(current_block.name, 0) + 1
        status_color = GREEN if cost != IMPASSABLE_COST else RED
        # Print detailed info for the cell (newline from previous progress update)
        type_text(f"\n{status_color}{BOLD}--- Sector ({r:02}, {c:02}) ---{RESET}", detail_char_delay)
        type_text(f"  {MAGENTA}Type:{RESET} {current_block.name}", detail_char_delay)
        type_text(f"  {MAGENTA}Desc:{RESET} {current_block.description}", detail_char_delay)

        
        status_text = f"Cost: {cost:.1f}" if cost != IMPASSABLE_COST else "IMPASSABLE"
        type_text(f"  {MAGENTA}Status:{RESET} {status_color}{status_text}{RESET}", detail_char_delay)
        type_text(f"  {MAGENTA}Stability:{RESET} {current_block.stability:.2f} (Required > {MIN_STABILITY_FOR_TRAVERSAL:.2f})", detail_char_delay)
        type_text(f"  {MAGENTA}Obstacle:{RESET} {current_block.is_obstacle}", detail_char_delay)

        # Vehicle specific observation
        if current_block.name == "Rocky Terrain":
            type_text(f"  {CYAN}Harvester Report:{RESET} High density mineral deposits detected. Potential resource extraction site.", detail_char_delay)
        elif current_block.name == "Ocean Abyss":
            type_text(f"  {CYAN}Aqua-Rover Report:{RESET} Extreme pressure gradient detected. Suggesting deeper reconnaissance for anomaly.", detail_char_delay)
        elif current_block.name == "Dense Fog":
            type_text(f"  {CYAN}Sky-Scout Report:{RESET} Localized sensor interference due to atmospheric particulate. Visual impaired.", detail_char_delay)
        elif current_block.name == "Volcanic Crater":
             type_text(f"  {CYAN}Thermal Sensor:{RESET} Extreme heat signature detected. Avoid direct contact. Air transit highly recommended.", detail_char_delay)
        elif current_block.name == "Magnetic Anomaly":
             type_text(f"  {CYAN}System Warning:{RESET} Localized EM field disrupting drone instruments. Navigation compromised.", detail_char_delay)


        if current_block.geological_properties:
            type_text(f"  {MAGENTA}Geology:{RESET}", detail_char_delay)
            for prop, val in current_block.geological_properties.items():
                prop_color = MAGENTA
                if prop == 'water' and val > MAX_WATER_COMPOSITION_FOR_TRAVERSAL * 0.7:
                    prop_color = YELLOW
                if prop in ['rock', 'silicates']:
                    solid_sum = current_block.geological_properties.get('rock', 0.0) + current_block.geological_properties.get('silicates', 0.0)
                    if solid_sum > MAX_SOLID_COMPOSITION_FOR_TRAVERSAL * 0.7:
                         prop_color = YELLOW
                type_text(f"    - {prop_color}{prop.capitalize()}: {val:.2f}{RESET}", detail_char_delay)
        else:
            type_text(f"  {MAGENTA}Geology:{RESET} No specific geological data.", detail_char_delay)
        type_text("-" * 40, detail_char_delay)
        # No extra time.sleep here, the char_delay handles it

    time.sleep(0) # Larger pause between rows for readability

# Final progress update to ensure it's on a new line
type_text(f"\n{GREEN}[08:30:00 ZULU] {RESET}{BOLD}Full grid detailed scan complete and uploaded to Mother Vessel.{RESET}")
time.sleep(1.0)

type_text(f"\n{BLUE}--- GRID COMPOSITION SUMMARY ---{RESET}")
type_text(f"{MAGENTA}Total Cells Scanned: {total_cells}{RESET}")
type_text(f"{MAGENTA}Traversable Zones: {traversable_count} ({ (traversable_count / total_cells) * 100:.2f}%) {RESET}")
type_text(f"{MAGENTA}Impassable Hazards: {impassable_count} ({ (impassable_count / total_cells) * 100:.2f}%) {RESET}")
type_text(f"{MAGENTA}Identified Terrain Types:{RESET}")
for terrain_type, count in sorted(terrain_type_counts.items()):
    type_text(f"  - {terrain_type}: {count} cells", detail_char_delay)
time.sleep(1.0)


type_text(f"\n{BLUE}--- MISSION BRIEF: BEACON RETRIEVAL STRATEGY ---{RESET}")
time.sleep(1.0)
type_text(f"{YELLOW}[08:35:00 ZULU] {RESET}With comprehensive grid data, Pathfinder-1 is now equipped for precise pathfinding. Initiating high-level route planning for the beacon retrieval.")
time.sleep(1.0)
type_text(f"{YELLOW}[08:36:00 ZULU] {RESET}Analyzing traversal costs and impassable zones. Identifying potential choke points and alternative routes for the 'Harvester' and 'Aqua-Rover' specialized units.")
time.sleep(1.0)
type_text(f"{YELLOW}[08:37:00 ZULU] {RESET}Recommending a multi-pronged approach: 'Sky-Scout' for aerial observation, 'Harvester' for ground route clearance/recon, 'Aqua-Rover' for deep water traversal.")
type_text(f"{BLUE}--- END OF ACQUISITION LOG ---{RESET}")

[94

m--- POST-LAND OPERATIONS: GRID MAPPING INITIATED ---

[07:50:00 ZULU] Makeshift HQ fully established. Commencing high-resolution grid mapping.
[07:51:00 ZULU] Deploying 'Harvester' ground drone for Sector A. 'Sky-Scout' aerial unit deploying for Sector B overview. 'Aqua-Rover' standing by for hydrological mapping.
[07:55:00 ZULU] Establishing high-bandwidth uplink to 'Odyssey Prime' Mother Vessel for real-time data synchronization. Mother Vessel acknowledging data stream.

--- DETAILED GRID ACQUISITION REPORT (10x10) ---
Scanning and populating grid with detailed environmental data. Stand by for extensive data log...

Scan Progress: [                    ] 1.0% Complete
(00, 00): ERROR - Unknown Object Type! Skipping detailed scan.
Scan Progress: [                    ] 2.0% Complete
(00, 01): ERROR - Unknown Object Type! Skipping detailed scan.
Scan Progress: [                    ] 3.0% Complete
(00, 02): ERROR - Unknown Object Type! Skipping detailed scan.
Scan Progress: [            